In [12]:
import numpy as np
import cv2 

In [13]:
#load the image 
img = cv2.imread('./m1/happy_parents.jpg')


In [15]:
cv2.imshow('happy parent', img)
cv2.waitKey()
cv2.destroyAllWindows()

In [16]:
#load model
face_detection_model =cv2.dnn.readNetFromCaffe('./model/deploy.prototxt.txt','./model/res10_300x300_ssd_iter_140000_fp16.caffemodel')


In [17]:
# blob form image
image= img.copy()
blob = cv2.dnn.blobFromImage(image,1,(300,300),(104,177,123),swapRB=True)
face_detection_model.setInput(blob)
detections= face_detection_model.forward()

In [18]:
detections

array([[[[0.        , 1.        , 0.89241487, ..., 0.23449695,
          0.49035037, 0.35220194],
         [0.        , 1.        , 0.8231071 , ..., 0.38387397,
          0.4965028 , 0.5540676 ],
         [0.        , 1.        , 0.10900012, ..., 0.4561051 ,
          0.4984677 , 0.61265206],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [19]:
detections.shape

(1, 1, 200, 7)

In [20]:
for i in range(detections.shape[2]):
    confidence= detections[0,0,i,2]
    if confidence >0.5:
        print(confidence)

0.89241487
0.8231071


In [21]:
#get height and withd of box
image = img.copy()
h,w = image.shape[:2]
for i in range(detections.shape[2]):
    confidence = detections[0,0,i,2]
    if confidence >0.5:
        
        #box= detections[0,0,i,3:7]
        #denormanlise arr to get actuall height and width
        box = detections[0,0,i,3:7]*np.array([w,h,w,h])
        box= box.astype(int)
        cv2.rectangle(image,(box[0],box[1]),(box[2],box[3]),(0,0,255))
        print(box)

cv2.imshow('face detection',image)
cv2.waitKey()
cv2.destroyAllWindows()

[427 156 490 234]
[411 255 496 369]


In [22]:
#step2 mask image
#get height and withd of box
image= img.copy()
h,w = image.shape[:2]
#mask image 
face_mask = np.zeros(image.shape[:2],dtype='uint8')# 2-d array graysclae


for i in range(detections.shape[2]):
    confidence = detections[0,0,i,2]
    if confidence >0.5:
        
        #box= detections[0,0,i,3:7]
        #denormanlise arr to get actuall height and width
        box = detections[0,0,i,3:7]*np.array([w,h,w,h])
        box= box.astype(int)
        cv2.rectangle(face_mask,(box[0],box[1]),(box[2],box[3]),255,-1)
        print(box)
        
inv_face_mask = cv2.bitwise_not(face_mask)
#step 3: apply mask imate to original image

bg_img= cv2.bitwise_and(image,image,mask=face_mask)
fg_img= cv2.bitwise_and(image,image,mask=inv_face_mask)

#step 4: blur image with gausian blur
bg_img_blur = cv2.GaussianBlur(bg_img,(51,51),0)

#Step 5: Addition operation 
face_blur = cv2.add(fg_img,bg_img_blur)

# display two image
cv2.imshow('origina',image)
#cv2.imshow('face detection',face_mask)
#cv2.imshow('face etection',inv_face_mask)
#cv2.imshow('backgrounf',bg_img)
#cv2.imshow('forre gorun image',fg_img)
#cv2.imshow('blur image', bg_img_blur
cv2.imshow('facial blur image', face_blur)
cv2.waitKey()
cv2.destroyAllWindows()



[427 156 490 234]
[411 255 496 369]


In [28]:
#cteate function
import numpy as np 
import cv2
def face_blur(img):
    #step2 mask image
    #get height and withd of box
    image= img.copy()
    blob = cv2.dnn.blobFromImage(image,1,(300,300),(104,177,123),swapRB=True)
    face_detection_model.setInput(blob)
    detections= face_detection_model.forward()
    h,w = image.shape[:2]
    #mask image 
    face_mask = np.zeros(image.shape[:2],dtype='uint8')# 2-d array graysclae
    
    
    for i in range(detections.shape[2]):
        confidence = detections[0,0,i,2]
        if confidence >0.5:
            
            #box= detections[0,0,i,3:7]
            #denormanlise arr to get actuall height and width
            box = detections[0,0,i,3:7]*np.array([w,h,w,h])
            box= box.astype(int)
            cv2.rectangle(face_mask,(box[0],box[1]),(box[2],box[3]),255,-1)
            print(box)
            
    inv_face_mask = cv2.bitwise_not(face_mask)
    #step 3: apply mask imate to original image
    
    bg_img= cv2.bitwise_and(image,image,mask=face_mask)
    fg_img= cv2.bitwise_and(image,image,mask=inv_face_mask)
    
    #step 4: blur image with gausian blur
    bg_img_blur = cv2.GaussianBlur(bg_img,(51,51),0)
    
    #Step 5: Addition operation 
    face_blur = cv2.add(fg_img,bg_img_blur)
    return face_blur


In [29]:
blur_img = face_blur(img)
cv2.imshow('original', img)
cv2.imshow('facial blur image', blur_img)
cv2.waitKey()
cv2.destroyAllWindows()


[427 156 490 234]
[411 255 496 369]


In [7]:
#real time facial blur

In [30]:
cap = cv2.VideoCapture(0)
while True: 
    ret, frame = cap.read()
    if ret==False:
        break
    blur_img=face_blur(frame)
    cv2.imshow('original',frame)
    cv2.imshow('blur image',blur_img)
    if cv2.waitKey(1)==27: # use esc to close program
        break

cap.release()
cv2.destroyAllWindows()
    

[259 101 424 308]
[257 102 425 310]
[257 102 425 310]
[257 103 426 309]
[257 103 425 309]
[257 103 425 309]
[257 102 426 309]
[257 102 426 309]
[256 102 426 309]
[256 102 426 309]
[255 102 426 309]
[255 102 426 309]
[256 102 426 308]
[256 102 426 308]
[257 101 425 309]
[257 101 425 309]
[257 101 425 309]
[258 100 426 309]
[258 100 426 309]
[258  97 427 308]
[258  97 427 308]
[260  99 429 307]
[260  99 429 307]
[262  99 431 307]
[262  99 431 307]
[264  99 432 308]
[264  99 432 308]
[266 102 436 306]
[267 103 437 304]
[267 103 437 304]
[268 104 438 305]
[268 104 438 305]
[268 103 438 306]
[268 103 438 306]
[267 105 438 307]
[267 105 438 307]
[268 104 437 306]
[268 104 437 306]
[267 102 438 305]
[267 102 438 305]
[266 103 436 306]
[266 103 436 307]
[266 103 436 307]
[266 102 435 306]
[266 102 435 306]
[265 101 433 306]
[265 101 433 306]
[264  93 429 305]
[264  93 429 305]
[260  98 430 306]
[260  98 430 306]
[257  96 423 302]
[256  97 423 302]
[256  97 423 302]
[255  96 421 301]
[255  96 4